# MongoDB VCore RAG


<img src = "./cosmosdbrag.png">

<img src= "./components.png">

### Installing Dependecies

In [ ]:
%pip install langchain==0.3.4
%pip install langchain-openai==0.2.3
%pip install pymongo==4.10.1


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

### Initialize OpenAI Client


In [ ]:
import openai
openai.api_type = "azure"
openai.base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")


### Initialize MongoDB Client

In [ ]:
from urllib.parse import quote_plus
from pymongo import MongoClient
mongo_connection_string = os.getenv("AZURE_COSMOS_CONNECTION_STRING")
mongo_username = quote_plus(os.getenv("AZURE_COSMOS_USERNAME"))
mongo_password = quote_plus(os.getenv("AZURE_COSMOS_PASSWORD"))


collection_name = os.getenv("AZURE_COSMOS_COLLECTION_NAME", "collectionName")
database_name = os.getenv("AZURE_COSMOS_DATABASE_NAME", "DatabaseName")

mongo_connection_string = mongo_connection_string.replace("<user>", mongo_username).replace(
    "<password>", mongo_password
)

# Initialize the MongoClient
mongo_client = MongoClient(mongo_connection_string)

# Create the database if it doesn't exist
db = mongo_client[database_name]

# Create the collection if it doesn't exist
collection = db[collection_name]

### Load Json Data

In [ ]:
import json
from langchain.docstore.document import Document

source_file_name = "./food_items.json"

def read_data(file_path) -> list[Document]:
    with open(file_path) as file:
        json_data = json.load(file)
        
    documents = []
    absolute_path = os.path.abspath(file_path)
    for idx, item in enumerate(json_data):
         documents.append(
            Document(page_content=json.dumps(item), metadata={"source": absolute_path, "seq_num": idx + 1})
        )
    
    return documents



In [ ]:
json_data = read_data("./food_items.json") 

In [ ]:
print(json_data[0])

### Initialize Embeddings Client

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

openai_embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME")
openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")

azure_openai_embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    model = openai_embeddings_model,
    azure_deployment = openai_embeddings_deployment
)

### Generate and Save Embeddings to MongoDB

In [ ]:
from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

index_name = os.getenv("AZURE_COSMOS_INDEX_NAME", "indexName")

vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_documents(
    json_data,
    azure_openai_embeddings,
    collection=collection,
    index_name=index_name
)

In [ ]:
from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch

# Run this to connect to the vector store
vector_store: AzureCosmosDBVectorSearch = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string=mongo_connection_string,
    namespace=f"{database_name}.{collection_name}",
    embedding=azure_openai_embeddings,
)

### Create an IVF Vector Index

In [ ]:
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)

# Read more about these variables in detail here. https://learn.microsoft.com/en-us/azure/cosmos-db/mongodb/vcore/vector-search
num_lists = 100
dimensions = 1536
similarity_algorithm = CosmosDBSimilarityType.COS
kind = CosmosDBVectorSearchType.VECTOR_IVF
m = 16
ef_construction = 64

# Create the collection and the index
vector_store.create_index(num_lists, dimensions, similarity_algorithm, kind, m, ef_construction)

### Test Vector Similarity on a query

In [ ]:
query = "Beef Bacon"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

## Bringing RAG to Life!!!!

### Defining RAG Workflow with help of a python function


In [ ]:
from openai import AzureOpenAI

azureOpenAIClient = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-05-01-preview"
)

GPT_model = os.getenv("AZURE_OPENAI_GPT_MODEL_NAME")

In [ ]:
def RAG(query: str) -> str:
    docs = vector_store.similarity_search(query)
    print(docs)
    context = docs[0].page_content
    
    print(context)
    
    system_prompt = (
        "you are a cooking assistant meant to answer the user query \n"
        "you will be given context to the prompt based upon which you have to answer the user query \n"
        "the context of the prompt has been taken from an index created in Azure MongoDB vCore API \n"
        "the schema for the context is as follows: \n"
        "category: string (the category to which the dish belongs) \n"
        "name: string (the name of the dish) \n"
        "description: string (the description about the dish) \n"
        "price: string (the price of the dish in USD) \n"
        "---------------------------------------------- \n"
        "strictly answer the user query from the context provided \n"
    )
    
    user_prompt = (
        f"user query: {query} \n"
        "answer the user query from the context provided \n"
        "context: " + context
    )
    
    response = azureOpenAIClient.chat.completions.create(
        model="gpt-4",  # model = "deployment_name".
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return str(response.choices[0].message.content)
    
    
    

In [ ]:
response = RAG("is capuccino available and at what price?")
print(response)